In [112]:
# Parameters
trx = "A"
mg = "B"
segments = [f"TRX{trx}",f"MG{mg}"]
#nprocs = 4

In [113]:
import sys

In [114]:
sys.path
sys.path.append('/home/yvonne/Desktop/docking_result')

In [115]:
sys.path

['/home/yvonne/Desktop/docking_result/5.dna_pli',
 '/home/yvonne/miniconda3/envs/edaf/lib/python39.zip',
 '/home/yvonne/miniconda3/envs/edaf/lib/python3.9',
 '/home/yvonne/miniconda3/envs/edaf/lib/python3.9/lib-dynload',
 '',
 '/home/yvonne/miniconda3/envs/edaf/lib/python3.9/site-packages',
 '/home/yvonne/Desktop/docking_result',
 '/home/yvonne/Desktop/docking_result',
 '/home/yvonne/Desktop/docking_result',
 '/home/yvonne/Desktop/docking_result',
 '/home/yvonne/Desktop/docking_result',
 '/home/yvonne/Desktop/docking_result',
 '/home/yvonne/Desktop/docking_result']

In [116]:
!pwd

/home/yvonne/Desktop/docking_result/5.dna_pli


In [117]:
from numpy import exp
import gzip
import shutil
import subprocess
import re
import pandas as pd
import time
import os
import os.path
from multiprocessing.pool import ThreadPool
from openbabel import pybel
import tarfile
import glob
import shutil
from rdkit.Chem import PandasTools
from collections import OrderedDict
from PLI import amino_acid, analysis, interaction, license, io

In [118]:
file = "guanA"
pro_n = "g_trxa"
lig_n = "g_dnab"

In [119]:
root_folder = "/home/yvonne/Desktop/docking_result/5.dna_pli"
root_file = f'{file}_pli'
rep_file = f"{pro_n}.pdb"
lig_file = f"{lig_n}.sdf"
i = "input"
o = "output"
l = "log"

In [120]:
i_path=f'{root_folder}/{root_file}/{i}'
o_path=f'{root_folder}/{root_file}/{o}'
l_path=f'{root_folder}/{root_file}/{l}'
receptor=f'{root_folder}/{root_file}/{rep_file}'
ligand_file=f'{root_folder}/{root_file}/{lig_file}'

In [121]:
ligand_file

'/home/yvonne/Desktop/docking_result/5.dna_pli/guanA_pli/g_dnab.sdf'

In [122]:
os.makedirs(i_path, exist_ok=True)   #
os.makedirs(o_path, exist_ok = True)  # 
os.makedirs(l_path, exist_ok = True)     #
## Get sdf stuff
ligands = PandasTools.LoadSDF(ligand_file, molColName=None) # Do not show Mol figure 

# Convert sdf to pdbqt
ligandlist = []
for m in pybel.readfile("sdf", ligand_file):
        t0 = m.data["Mode"]
        m.write("pdbqt", f"{i_path}/{m.title}_{t0}.pdbqt", overwrite=True)
        ligandlist.append(f"{m.title}_{t0}")
ligands = ligands[["ID", "Mode", "Score", "RMSD_LB", "RMSD_UB"]]

In [123]:
ligands

,ID,Mode,Score,RMSD_LB,RMSD_UB
0,g_dnab.pdb,1,0,0.0,0.0


In [124]:
segid = segments
prot = amino_acid.read_protein_all_atom_model(receptor, segid)

intro_message = "Date: {}   Creator: {}\n".format(time.asctime(), os.getenv('USER'))
intro_message += license.intro()


##
t0 = []
t1 = []
t2 = []
t3 = []
t4 = []
t5 = []
for r in prot.residues:
    segid = r.segid
    resid = r.resid
    resn = r.resn
    for a in r.atoms:
        id = a.id  ## 2022/08/16 amended by Bruce
        name = a.name 
        type = a.type
        t0.append(segid)
        t1.append(resid)
        t2.append(resn)
        t3.append(name)
        t4.append(type)
        t5.append(id)
atomlist = pd.DataFrame([t5, t0, t1, t2, t3, t4], index=["id", "segid", "resid", "resn", "name", "type"]).T

/home/yvonne/miniconda3/envs/edaf/lib/python3.9/site-packages/MDAnalysis/topology/PDBParser.py:345: UserWarning: Unknown element  found for some atoms. These have been given an empty element record. If needed they can be guessed using MDAnalysis.topology.guessers.
  warnings.warn(wmsg)
/home/yvonne/miniconda3/envs/edaf/lib/python3.9/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: 
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/yvonne/miniconda3/envs/edaf/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:432: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"


In [125]:
atomlist

,id,segid,resid,resn,name,type
0,1,MGB,1,MG,MG,Met
1,2,MGB,2,MG,MG,Met
2,3,TRXA,1,MET,N,NH3
3,4,TRXA,1,MET,CA,CT1
4,5,TRXA,1,MET,C,C
...,...,...,...,...,...,...
3595,3601,TRXA,236,ALA,HN,H
3596,3602,TRXA,236,ALA,HA,CT1
3597,3603,TRXA,236,ALA,HB1,HA3
3598,3604,TRXA,236,ALA,HB2,HA3


In [126]:
d_rid_table = dict()
for row in atomlist.iterrows():
    key = (row[1]['segid'], row[1]['resid'], row[1]['resn'], row[1]['name'])
    d_rid_table[key] =  row[1]['id']

In [127]:
ligAtomList = []
interactionList = []
for ligand in ligandlist:
    input = f'{i_path}/{ligand}.pdbqt'
    output = f'{l_path}/{ligand}.csv'
    log = f'{l_path}/{ligand}.log'
    # log = os.path.join(root_folder, l, f'{ligand}.log')
    # input = os.path.abspath(f"input/{ligand}.pdbqt")
    # output = os.path.abspath(f"output/{ligand}.csv")
    # log = os.path.abspath(f"log/{ligand}.log")
    
    
    # file = open(log, 'w', encoding='UTF-8')
    file = open(log, 'w', encoding='UTF-8')       #amend
    lig = amino_acid.read_atom_group(input)

    main_message = ""
    all_interaction = []

    # Calculate steric interaction
    steric_main, steric_side, msg = interaction.steric_interaction(prot, lig, log=True)
    all_interaction.append(steric_main)
    all_interaction.append(steric_side)
    main_message += msg

    # Calculate hydrophobic interaction
    hydrophobic_main, hydrophobic_side, msg = interaction.hydrophobic_interaction(prot, lig, log=True)
    all_interaction.append(hydrophobic_main)
    all_interaction.append(hydrophobic_side)
    main_message += msg

    # Calculate hbond interaction
    hbond_main, hbond_side, msg = interaction.hbond_interaction(prot, lig, log=True)
    all_interaction.append(hbond_main)
    all_interaction.append(hbond_side)
    main_message += msg

    all_interaction.reverse()
    features = OrderedDict()
    
    # Generate interaction table
    for index, types in enumerate(["steric", "hydrophobic", "hbond"]):
        for pose in interaction.list_of_interaction()[types]:
            features[pose] = all_interaction.pop()
            
    table = analysis.create_feature_table(**features)
    table.to_csv(output, float_format='%.5f', index=False)
    
    main_message += "\n\nDate: {}\n".format(time.asctime())
    file.write(intro_message + main_message)
    file.close()
    
    #
    t1 = []
    t3 = []
    t4 = []
    t5 = []

    for a in lig.atoms:
        id = int(a.index) 
        name = a.name 
        type = a.type
        t3.append(name)
        t4.append(type)
        t5.append(id)
    t1 = pd.DataFrame([t5, t3, t4], index=["id", "name", "type"]).T
    ligAtomList.append(t1)
    
    #
    st = table.filter(like="Steric").sum(axis=1)
    hy = table.filter(like="Hydrophobic").sum(axis=1)
    hb = table.filter(like="H-bond").sum(axis=1)
    surface_d = table.Surface_d
    lid = ((table["PairJ"].str.extract(r'.*[a-zA-Z]([0-9]+)')).astype(int) + 1)[0]
    t = zip(list(table["Segid"]), list(table["Resid"]), list(table["Resn"]), list(table["PairI"]))
    
    #ammemded by yizao
    rid = list()
    for x in t:
        key = (x[0], x[1], x[2], x[3])
        rid.append(d_rid_table[key])
        
        #if (x[0] == 'TRXB') & (x[1]==14) & (x[2]=='ASP') & (x[3]=='OD1'):
        #    print(d_rid_table[key])
    #rid = pd.DataFrame(rid)[0]  #####Something really wrong here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    
    d_result = {'lid': lid, 'rid': rid, 'st': st, 'hy': hy, 'hb': hb, 'surface_d': surface_d}
    df_final = pd.DataFrame(d_result)
    interactionList.append(df_final)
    
    #rid = range(len(lid))
    #rid = pd.DataFrame([int(atomlist[(atomlist["segid"] == x[0]) & (atomlist["resid"] == x[1]) & (atomlist["name"] == x[2])].head(1)["id"]) for x in t])[0]
    #interactionList.append(pd.DataFrame([lid, rid, st, hy, hb, surface_d], index=["lid", "rid", "st", "hy", "hb", "surface_d"]).T)

/home/yvonne/miniconda3/envs/edaf/lib/python3.9/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/yvonne/miniconda3/envs/edaf/lib/python3.9/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: HD
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/yvonne/miniconda3/envs/edaf/lib/python3.9/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: OA
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


In [128]:
store = pd.HDFStore(f"{root_folder}/{root_file}/score.h5")
store["receptor/atoms"] = atomlist

for (index, row), m, n in zip(ligands.iterrows(), ligAtomList, interactionList):
    t = "ligands/" + str(row["ID"] + "," + str(row["Mode"]))
    store[t + "/atoms"] = m
    store[t + "/interaction"] = n

store.keys()
store.close()

<ipython-input-128-02f01ba132ad>:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['id', 'segid', 'resid', 'resn', 'name', 'type'], dtype='object')]

  store["receptor/atoms"] = atomlist
/home/yvonne/miniconda3/envs/edaf/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'g_dnab.pdb,1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
<ipython-input-128-02f01ba132ad>:6: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['id', 'name', 'type'], dtype='object')]

  store[t + "/atoms"] = m


In [18]:
# # Archive log files
# tar = tarfile.open("log.tar.gz", "w:gz")
# tar.add("log", arcname=os.path.basename("log"))
# tar.close()

In [19]:
#shutil.rmtree("input")
#shutil.rmtree("output")
#shutil.rmtree("log")
#os.remove("receptor.pdb")
#os.remove("ligands.sdf")